In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import onnx
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx_coreml  # onnx-coreml 라이브러리 사용

# 샘플 사용자 습관 데이터 생성
data = {
    'habit_name': ['명상', '운동', '독서', '물 마시기', '일기 쓰기', '요가', '감사일기', '달리기', '요리', '청소'],
    'completed_count': [5, 10, 3, 8, 4, 2, 7, 6, 9, 1],
    'average_time_spent': [10, 45, 20, 5, 15, 30, 12, 50, 40, 8],  # 분 단위
    'preferred_time_of_day': ['아침', '저녁', '아침', '오후', '저녁', '오전', '오전', '오후', '저녁', '아침'],
    'habit_category': ['건강', '운동', '지식', '건강', '마음', '운동', '마음', '운동', '생활', '생활'],
    'recommended': [1, 1, 0, 1, 0, 0, 1, 1, 1, 0]
}

df = pd.DataFrame(data)

# 문자열 데이터를 숫자로 변환
label_encoders = {}
for col in ['preferred_time_of_day', 'habit_category']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 특성과 라벨 분리
X = df[['completed_count', 'average_time_spent', 'preferred_time_of_day', 'habit_category']]
y = df['recommended']

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# ✅ ONNX 모델 변환
initial_type = [('input', FloatTensorType([None, X.shape[1]]))]
onnx_model = convert_sklearn(model, initial_types=initial_type)

# ONNX 모델 저장
onnx_model_path = "HabitiQ.onnx"
onnx.save_model(onnx_model, onnx_model_path)

# ✅ Core ML 모델 변환 (onnx-coreml 사용)
# ONNX 모델 로드
onnx_model = onnx.load(onnx_model_path)

# ONNX 모델을 Core ML 모델로 변환
coreml_model = onnx_coreml.convert(onnx_model)

# Core ML 모델 저장
coreml_model.save("HabitiQ.mlmodel")

print("✅ Core ML 변환 완료! HabitiQ.mlmodel 저장됨.")

ModuleNotFoundError: No module named 'onnx'

In [15]:
pip install onnx

  Using cached onnx-1.17.0-cp312-cp312-macosx_12_0_universal2.whl.metadata (16 kB)
Using cached onnx-1.17.0-cp312-cp312-macosx_12_0_universal2.whl (16.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install skl2onnx


  Using cached skl2onnx-1.18.0-py2.py3-none-any.whl.metadata (3.2 kB)
Using cached skl2onnx-1.18.0-py2.py3-none-any.whl (300 kB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install protobuf==3.20.2
pip install coremltools
pip install onnx-coreml
pip install onnxconverter-common

SyntaxError: invalid syntax (541319849.py, line 1)